In [1]:
import webvtt
import pandas as pd
import json
import os

In [9]:
def vtt2json(usage:str):
    files = os.listdir(f'./data/lnps/{usage}')
    communication_list = []
    for file in files:
        current_speaker = None
        old_speaker = "Tim Pritlo"
        current_text = []
        speaker_list = ['Tim Pritlo']
        if os.path.isfile(f'./data/lnps/{usage}/{file}'):
            for caption in webvtt.read(f'./data/lnps/{usage}/{file}'):
                if "Neumann" in caption.raw_text:
                    current_speaker = "Linus Neumann"
                elif "Pritlove" in caption.raw_text:
                    current_speaker = "Tim Pritlo"
                if old_speaker == current_speaker:
                    current_text.append(caption.text)
                else:
                    speaker_list.append(current_speaker)
                    communication_list.append(' '.join(current_text))
                    current_text = []
                    current_text.append(caption.text)
                    old_speaker = caption.raw_text.split('>')[0].split('<')[1].split('v')[1].strip()
            communication_list.append(' '.join(current_text))
            json_obj = []
            for elm in range(0,int(len(communication_list)/2),2):
                json_obj.append({'input': communication_list[elm], 
                                'output': communication_list[elm+1],
                                'instruction': ' ',
                                'introduction':' '}
                               )
            with open(f'./data/linus/{usage}/linus_data_{file}.json', 'w') as f:
                json.dump(json_obj, f)

            json_obj = []
            for elm in range(1,int(len(communication_list)/2),2):
                json_obj.append({'input': communication_list[elm], 
                            'output': communication_list[elm+1],
                                'instruction': ' ',
                                'introduction':' '})
            with open(f'./data/tim/{usage}/tim_data_train_{usage}_{file}.json', 'w') as f:
                json.dump(json_obj, f)

In [10]:
vtt2json('validate')

In [3]:
def merge_JsonFiles(name:str,usage:str):
    result = list()
    files=os.listdir(f'./data/{name}/{usage}')
    for f1 in files:
        if name in f1:
            with open(f'./data/{name}/{f1}', 'r') as infile:
                result.extend(json.loads(infile.read()))

    with open(f'./data/{name}/{name}_{usage}_data.json', 'w') as output_file:
        json.dump(result, output_file)

merge_JsonFiles(files,"linus")

In [ ]:
def json2lines(file,name:str,usage:str):
    with open(f'./data/{name}/{name}_{usage}_data.jsonl', 'w') as outfile:
        for entry in file:
            json.dump(entry, outfile)
            outfile.write('\n')
json2lines(contents,'linus','validate')

In [19]:
data = []
with open(f'./data/linus/linus_data.jsonl', 'r') as file:
    for line in file:
        data.append(json.loads(line))

In [21]:
json_obj = []
for elm in data:
    json_obj.append({'input':elm['input'], 
                    'output': elm['output'],
                                'instruction': ' ',
                                'introduction':' '})

In [25]:
with open('./data/linus/linus_data.json', 'r') as j:
     contents = json.loads(j.read())

In [27]:
with open(f'./data/linus/linus_trin_data.jsonl', 'w') as outfile:
    for entry in contents:
        json.dump(entry, outfile)
        outfile.write('\n')

In [23]:
with open(f'./data/linus/linus_data.json', 'w') as f:
        json.dump(json_obj, f)

In [15]:
with open(f'./data/linus/linus_data.jsonl', 'r') as infile:
    data = json.loads(infile.read())

JSONDecodeError: Extra data: line 2 column 1 (char 53)

In [11]:
with open('./data/linus/validate/linus_data_lnp418-impulsgeber-eines-modernen-verwaltungsmanagements.transcript.vtt.json', 'r') as j:
     contents = json.loads(j.read())

In [12]:
def json2lines(file,name:str,usage:str):
    with open(f'./data/{name}/{name}_{usage}_data.jsonl', 'w') as outfile:
        for entry in file:
            json.dump(entry, outfile)
            outfile.write('\n')
json2lines(contents,'linus','validate')

In [2]:
df = pd.read_json('./data/linus/linus_data.json')

In [3]:
df.head()

,introduction,input,output
0,Guten Morgen Linus.,,Guten.
1,"Sag mal, wie steht eigentlich der Bitcoin scho...",,Ist noch Luft nach unten.
2,"Logbuch, Netzpolitik Nummer vierhundertundfünf...",,Am Ende nicht aktiviert hast.
3,"Die ich dann irgendwie, wo ich irgendwie das f...",,"Habe ich dahin gefahren Tim, also insofern der..."
4,"Na ja, ich äh konnte das dann nicht auf mir si...",,Das war bestimmt eine schöne Arbeit.


In [4]:
df['length'] = df['output'].apply(
    lambda row: min(len(row.split(" ")), len(row)) if isinstance(row, str) else None
)

In [5]:
tmp = df[df['length'].between(30,1000) ]

In [16]:
#df.to_json('./data/linus/linus_data_small.json')

In [6]:
json_obj = []
for index, elm in tmp.iterrows():
    json_obj.append({'instruction': elm['introduction'], 
                    'input':'',
                    'output': elm['output']})
with open(f'./data/linus/linus_data_small.json', 'w') as f:
    json.dump(json_obj, f)

In [8]:
df = pd.read_json("./data/linus/linus_data_small.json")

In [20]:
test = 'Was machst du eigentlich beruflich?'

In [26]:
len(test.split('?'))

2

In [40]:
counter = 0
for index, elm in df.iterrows():
    if "?" in elm['instruction']:
        elm['instruction'] = elm['instruction'].split('?')[0].join(' ?')
        counter +=1

counter

1075

In [39]:
json_obj = []
for index, elm in df.iterrows():
    json_obj.append({'instruction': elm['instruction'], 
                    'input':'',
                    'output': elm['output']})
with open(f'./data/linus/linus_data_small_questions.json', 'w') as f:
    json.dump(json_obj, f)

In [29]:
from datasets import load_dataset

train_dataset = load_dataset('json', data_files='./data/linus/linus_validate_data.jsonl', split='train') 

In [70]:
for caption in webvtt.read('./data/lnp463-lass-uns-die-guten-spielen.transcript.vtt'):
        print(caption.start)
        print(caption.raw_text)
        #print(caption.end)
        #if 'Linus' in caption.raw_text:
        #    print(caption.raw_text)

00:00:00.576
<v Tim Pritlove>Guten Morgen Linus.
00:00:02.297
<v Linus Neumann>Guten Morgen Tim.
00:00:03.877
<v Tim Pritlove>So jetzt wo Apple mitspielt ist ja Augmented Reality eine echte Sache.
00:00:09.880
<v Tim Pritlove>Ich hab ja manchmal Probleme mit der Realität und frage mich wie man die jetzt
00:00:13.602
<v Tim Pritlove>anreichen könnte damit es besser wird.
00:00:16.823
<v Linus Neumann>Also ich sag ganz ehrlich um um diese
00:00:19.664
<v Linus Neumann>Realität zu verbessern sehe ich für Augmentation einfach keine Zukunft.
00:00:45.277
<v Tim Pritlove>Musik Logbuch Netzpolitik Nummer 463, so sieht es aus, genau. Vom 20.
00:00:54.123
<v Tim Pritlove>Juni 2023, hallo wir sind's, Tim und Linus. Wir sagen nie wie wir heißen übrigens,
00:00:59.667
<v Tim Pritlove>ist dir das schonmal aufgefallen?
00:01:02.569
<v Tim Pritlove>Wir gehen mal davon aus, dass alle Bescheid wissen.
00:01:07.873
<v Tim Pritlove>Ist das angemessen?
00:01:09.654
<v Linus Neumann>Weiß ich nicht.
00:01:1

# 

# 

# 